In [60]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Permute
from keras.layers import ConvLSTM2D, Conv2D, MaxPooling2D
from keras.layers import MaxPooling1D, Conv1D
from keras.layers import GRU, LSTM, BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.layers import Activation

A01T = h5py.File('datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

a = data[:56]
b = data[57:]
data = np.vstack((a,b))
a = labels[:56]
b = labels[57:]
labels = np.hstack((a,b))
#enc = OneHotEncoder()
#enc_labels = enc.fit_transform(labels.reshape(-1,1)).toarray()
enc_labels = to_categorical(labels-769, num_classes=4)
print(enc_labels)

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [4]:
data2d = data.reshape(data.shape[0], data.shape[2], data.shape[1], 1)
print data2d.shape

(287, 1000, 25, 1)


In [6]:
bs, t, f = data.shape
np.random.seed(42)
shuffle = np.random.choice(bs,bs,replace=False)

#mask = np.ones_like(data)
#mask[[:,:,21:24]] = True
#newdata = data[mask]
#np.delete(mask, :,:,24)
train_samples = 200
train_data = data2d[shuffle[:train_samples],:,:]
train_labels = enc_labels[shuffle[:train_samples]]
test_data = data2d[shuffle[train_samples:],:,:]
test_labels = enc_labels[shuffle[train_samples:]]
print data2d.shape
print train_data.shape

(287, 1000, 25, 1)
(200, 1000, 25, 1)


In [94]:
model = Sequential()
model.add(Conv2D(15, kernel_size=(20,1), activation='relu', data_format='channels_last', input_shape=(1000, 25, 1)))  
model.add(Conv2D(15, kernel_size=(15,20), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(30, kernel_size=(20,15), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(60, kernel_size=(20,30), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(120, kernel_size=(20,60), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()
hist = model.fit(train_data,train_labels,epochs=10,validation_split=0.25,batch_size=32)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_258 (Conv2D)          (None, 981, 25, 15)       315       
_________________________________________________________________
conv2d_259 (Conv2D)          (None, 967, 6, 15)        67515     
_________________________________________________________________
max_pooling2d_133 (MaxPoolin (None, 322, 6, 15)        0         
_________________________________________________________________
permute_55 (Permute)         (None, 322, 15, 6)        0         
_________________________________________________________________
conv2d_260 (Conv2D)          (None, 303, 1, 30)        54030     
_________________________________________________________________
max_pooling2d_134 (MaxPoolin (None, 101, 1, 30)        0         
_________________________________________________________________
permute_56 (Permute)         (None, 101, 30, 1)        0         
__________